### Running Palace Simulations on GDSFactory+ Cloud


In [ ]:
!uv pip install "gplugins[palace] @ git+https://github.com/gdsfactory/gplugins.git@palace-api"

In [ ]:
from gdsfactoryplus import sim
from sim_utils import plot_mesh_pv, print_job_summary, upload_simulation_dir

In [ ]:
# === Configuration ===
OUTPUT_DIR = "./palace-sim-cpw"

### Load a pcell from IHP PDK

In [ ]:
import gdsfactory as gf

from ihp import LAYER, PDK

PDK.activate()


@gf.cell
def gsg_electrode(
    length: float = 100,
    s_width: float = 20,
    g_width: float = 40,
    gap_width: float = 15,
    layer=LAYER.TopMetal2drawing,
) -> gf.Component:
    """
    Create a GSG (Ground-Signal-Ground) electrode.

    Args:
        length: horizontal length of the electrodes
        s_width: width of the signal (center) electrode
        g_width: width of the ground electrodes
        gap_width: gap between signal and ground electrodes
        layer: layer for the metal
    """
    c = gf.Component()

    # Top ground electrode
    r1 = c << gf.c.rectangle((length, g_width), centered=True, layer=layer)
    r1.move((0, (g_width + s_width) / 2 + gap_width))

    # Center signal electrode
    _r2 = c << gf.c.rectangle((length, s_width), centered=True, layer=layer)

    # Bottom ground electrode
    r3 = c << gf.c.rectangle((length, g_width), centered=True, layer=layer)
    r3.move((0, -(g_width + s_width) / 2 - gap_width))

    # Add ports at the gaps
    c.add_port(
        name="P1",
        center=(-length / 2, -(s_width + gap_width) / 2),
        width=gap_width,
        orientation=0,
        port_type="electrical",
        layer=layer,
    )

    c.add_port(
        name="P2",
        center=(-length / 2, (s_width + gap_width) / 2),
        width=gap_width,
        orientation=0,
        port_type="electrical",
        layer=layer,
    )

    c.add_port(
        name="P3",
        center=(length / 2, (s_width + gap_width) / 2),
        width=gap_width,
        orientation=180,
        port_type="electrical",
        layer=layer,
    )

    c.add_port(
        name="P4",
        center=(length / 2, -(s_width + gap_width) / 2),
        width=gap_width,
        orientation=180,
        port_type="electrical",
        layer=layer,
    )

    return c


c = gsg_electrode()
cc = c.copy()
cc.draw_ports()
cc

### Generate the mesh

In [ ]:
from gplugins import palace_api as pa

# Transmission lines (no substrate)
stack = pa.get_stack(substrate_thickness=2.0, air_above=300.0)


# Left CPW port (2 elements)
pa.configure_cpw_port(
    port_upper=c.ports["P2"],
    port_lower=c.ports["P1"],
    layer="topmetal2",
    length=5.0,
)

# Right CPW port (2 elements)
pa.configure_cpw_port(
    port_upper=c.ports["P3"],
    port_lower=c.ports["P4"],
    layer="topmetal2",
    length=5.0,
)


# Extract → returns 2 CPWPort objects
ports = pa.extract_ports(c, stack)

# Generate mesh
result = pa.generate_mesh(
    component=c,
    stack=stack,
    ports=ports,
    output_dir=OUTPUT_DIR,
    config=None,
)

In [ ]:
# Static PNG
plot_mesh_pv(
    f"{OUTPUT_DIR}/palace.msh",
    output=f"{OUTPUT_DIR}/mesh.png",
    show_groups=["metal", "P"],
    interactive=False,
)

# Interactive
# plot_mesh_pv(f"{OUTPUT_DIR}/palace.msh", show_groups=['metal', 'P'], interactive=True)

### Upload the mesh and start the simulation job

In [ ]:
pre_job = upload_simulation_dir(OUTPUT_DIR, sim.JobDefinition.PALACE)
job = sim.start_simulation(pre_job)
finished_job = sim.wait_for_simulation(job)

In [ ]:
print_job_summary(finished_job)

In [ ]:
results = sim.download_results(finished_job)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(results["port-S.csv"])
df.columns = df.columns.str.strip()  # Remove whitespace from column names

freq = df["f (GHz)"]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))

# Magnitude plot
ax1.plot(freq, df["|S[1][1]| (dB)"], marker=".", label="S11")
ax1.plot(freq, df["|S[2][1]| (dB)"], marker=".", label="S21")
ax1.set_xlabel("Frequency (GHz)")
ax1.set_ylabel("Magnitude (dB)")
ax1.set_title("S-Parameters")
ax1.legend()
ax1.grid(True)

# Phase plot
ax2.plot(freq, df["arg(S[1][1]) (deg.)"], marker=".", label="S11")
ax2.plot(freq, df["arg(S[2][1]) (deg.)"], marker=".", label="S21")
ax2.set_xlabel("Frequency (GHz)")
ax2.set_ylabel("Phase (deg)")
ax2.legend()
ax2.grid(True)

plt.tight_layout()